In [1]:
import os
import re
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import markdownify
import OpenDartReader
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
api_key = os.environ["OPENDART_API_KEY"]
dart = OpenDartReader(api_key=api_key)

In [3]:
stock_code = "217190"
corp_name = "제너셈"

df_reports = dart.list(stock_code, kind="A")
df_reports = df_reports[df_reports["report_nm"].str.contains("사업보고서")]
df_reports

,corp_code,corp_name,stock_code,corp_cls,report_nm,rcept_no,flr_nm,rcept_dt,rm
1,00666204,제너셈,217190,K,사업보고서 (2023.12),20240320001808,제너셈,20240320,연
5,00666204,제너셈,217190,K,사업보고서 (2022.12),20230317000893,제너셈,20230317,연
9,00666204,제너셈,217190,K,사업보고서 (2021.12),20220318001019,제너셈,20220318,연
13,00666204,제너셈,217190,K,사업보고서 (2020.12),20210317001227,제너셈,20210317,
15,00666204,제너셈,217190,K,[기재정정]사업보고서 (2019.12),20210302003642,제너셈,20210302,
16,00666204,제너셈,217190,K,[기재정정]사업보고서 (2018.12),20210302003635,제너셈,20210302,
17,00666204,제너셈,217190,K,[기재정정]사업보고서 (2017.12),20210302003622,제너셈,20210302,
18,00666204,제너셈,217190,K,[기재정정]사업보고서 (2016.12),20210302003608,제너셈,20210302,
33,00666204,제너셈,217190,K,[기재정정]사업보고서 (2015.12),20160511000421,제너셈,20160511,


In [4]:
from dart_document_loader import DartDocumentLoader

rcept_no = df_reports.iloc[0]["rcept_no"]
loader = DartDocumentLoader(rcept_no)
docs = loader.load()

In [5]:
def create_folder(path):
    if not os.path.exists(path):
        os.makedirs(path)


def remove_non_letters(string):
    return "".join(char for char in string if char.isalpha())

In [9]:
folder_path = f".files/{stock_code}_{corp_name}"
create_folder(folder_path)

In [10]:
for doc in docs:
    title = remove_non_letters(doc.metadata["title"])
    section = remove_non_letters(doc.metadata["section"])
    index = doc.metadata["index"]
    markdown_content = doc.page_content

    file_name = f"{index}_{section}_{title}.md"
    file_name = f"{folder_path}/{file_name}"

    with open(file_name, "w") as f:
        f.write(markdown_content)